In [ ]:
%%writefile app.py
import os
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException
import pandas as pd
import chromadb
import uuid
import re
from langchain_community.document_loaders import WebBaseLoader

groq_api_key = 'Your_API_Key'

class Chain:

  def __init__(self):
    self.llm = ChatGroq(
    temperature=0.1,
    groq_api_key = 'Your_API_Key',
    model_name = 'llama-3.3-70b-versatile')

  def extract_jobs(self, clean_txt):
    prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """)

    prompt_chain = prompt_extract | self.llm
    res = prompt_chain.invoke(input={'page_data':clean_txt})

    try:
      json_parser = JsonOutputParser()
      res = json_parser.parse(res.content)
    except OutputParserException:
      raise OutputParserException('Context too big to handle.')
    return res if isinstance(res, list) else [res]

  def write_mail(self, job, links):
    prompt_mail = PromptTemplate.from_template('''
      ### JOB DESCRIPTION:
      {job_description}

      ### INSTRUCTION:
      You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
      the seamless integration of business processes through automated tools.
      Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
      process optimization, cost reduction, and heightened overall efficiency.
      Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
      in fulfilling their needs.
      Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
      Remember you are Mohan, BDE at AtliQ.
      Do not provide a preamble.
      ### EMAIL (NO PREAMBLE):
    ''')

    chain_mail = prompt_mail | self.llm
    res = chain_mail.invoke({'job_description': str(job), 'link_list': links})
    return res.content


class Portfolio:

  def __init__(self, file_path = 'my_portfolio.csv'):
    self.file_path = file_path
    self.data = pd.read_csv(self.file_path)
    self.chroma_client = chromadb.PersistentClient('vectorstores')
    self.new_collection = self.chroma_client.get_or_create_collection(name = 'portfolios')

  def load_portfolio(self):
      if not self.new_collection.count():
          for _, row in self.data.iterrows():
              self.new_collection.add(documents=row["Techstack"],
                                    metadatas={"links": row["Links"]},
                                    ids=[str(uuid.uuid4())])
  def query_links(self, skills):
    return self.new_collection.query(query_texts=skills, n_results=2).get('metadatas',[])


def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*?>', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)
    # Trim leading and trailing whitespace
    text = text.strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text


def create_streamlit(llm, portfolio, clean_text):

  st.title('Cold Mail Generator')
  url_input = st.text_input('Enter a valid URL here')
  submit_button = st.button('Submit')

  if submit_button:

    try:
      webload = WebBaseLoader([url_input])
      data = clean_text(webload.load().pop().page_content)
      portfolio.load_portfolio()
      jobs = llm.extract_jobs(data)
      for job in jobs:
        skills = job.get('skills',[])
        link_1 = portfolio.query_links(skills)
        mail_1 = llm.write_mail(job, link_1)
        st.code(mail_1,language='markdown')

    except Exception as e:
      st.error('An error occured: {e}')


if __name__ == "__main__":
    chain = Chain()
    portfolio = Portfolio()
    st.set_page_config(layout="wide", page_title="Cold Email Generator", page_icon="📧")
    create_streamlit(chain, portfolio, clean_text)





Overwriting app.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("334RUSAsc8UUBiEQbpQTLKgMbf8_3ykWUJFt7NvVMv4N1oVqp")

In [ ]:
# kill any previous server on 8501, then start Streamlit
!fuser -k 8501/tcp >/dev/null 2>&1 || true
import subprocess, time
p = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=true"])
time.sleep(3)  # give it a moment to boot

In [ ]:
from pyngrok import ngrok

# close any old tunnels and agent
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)
ngrok.kill()

public_url = ngrok.connect(8501, "http")
print("Streamlit app:", public_url)

Streamlit app: NgrokTunnel: "https://nonprominently-unpredisposing-burton.ngrok-free.dev" -> "http://localhost:8501"
